In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

# Translation

##  Data

In [2]:
import pandas as pd

In [3]:
combined_df_all = pd.read_csv('combined_df.csv', usecols=['EN', 'TI', 'EN_tokenized', 'TI_tokenized'])

In [4]:
combined_df_all.drop(columns=['EN_tokenized', 'TI_tokenized'], inplace=True)
combined_df_all["EN"] = combined_df_all["EN"].str.replace(',', '')
combined_df_all["TI"] = combined_df_all["TI"].str.replace(',', '')
combined_df_all.reset_index(drop=True, inplace=True)
combined_df_all.to_csv('dat_1k.csv')

In [5]:
bi_dat_2 = pd.read_csv('bi_dat_2.csv', usecols = ['TI', 'EN', 'TI_tokenized', 'EN_tokenized'])
combined_df = pd.read_csv('combined_df.csv', usecols=['EN', 'TI', 'EN_tokenized', 'TI_tokenized'])

In [6]:
dat_5k = pd.concat([bi_dat_2, combined_df])

In [7]:

dat_5k.reset_index(drop=True, inplace=True)

In [8]:
dat_5k = dat_5k.iloc[:,[1,0,2,3]]

In [9]:
dat_5k.drop(columns=['EN_tokenized', 'TI_tokenized'], inplace=True)
dat_5k["EN"] = dat_5k["EN"].str.replace(',', '')
dat_5k["TI"] = dat_5k["TI"].str.replace(',', '')

<ipython-input-9-47a3349666d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat_5k.drop(columns=['EN_tokenized', 'TI_tokenized'], inplace=True)
<ipython-input-9-47a3349666d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat_5k["EN"] = dat_5k["EN"].str.replace(',', '')
<ipython-input-9-47a3349666d9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [10]:
dat_5k.reset_index(drop=True, inplace=True)

In [11]:
dat_5k.to_csv('dat_5k.csv')

## Processing

In [13]:
import numpy as np
np.random.seed(1)
text_file = 'dat_1k.csv'
with open(text_file) as f:
    lines = f.read().split('\n')[:-1]

prefix = "translate English to Tigrinya: "
text_pairs = []
for line in lines:
    ind,orig, target = line.split(',')
    orig = orig.replace('"', '')
    target = target.replace('"', '')
    text_pairs.append({'orig': orig, 'target': target})

#Let's create some splits
np.random.shuffle(text_pairs)
num_valid_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_valid_samples
train_pairs = text_pairs[:num_train_samples]
valid_pairs = text_pairs[num_train_samples : num_train_samples + num_valid_samples]
test_pairs = text_pairs[num_train_samples + num_valid_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(valid_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

1180 total pairs
826 training pairs
177 validation pairs
177 test pairs


In [14]:
train_pairs[0]

{'orig': 'But you shall go to my country and to my relatives and take a wife for my son Isaac.',
 'target': 'ናብ ሃገረይን ናብ ዓሌተይን ኬድካ ደኣ ንወደይ ይስሃቅ ሰበይቲ ኣምጽኣሉ፡ በሎ።'}

In [15]:
train_list = []

In [16]:
for i in range(len(train_pairs)):
  train_list.append({'id': i, 'translation': {'en': train_pairs[i]['orig'],'ti':train_pairs[i]['target']}})

In [17]:
val_list = []
test_list = []

for i in range(len(valid_pairs)):
  val_list.append({'id': i, 'translation': {'en': valid_pairs[i]['orig'],'ti':valid_pairs[i]['target']}})

for i in range(len(test_pairs)):
  test_list.append({'id': i, 'translation': {'en': test_pairs[i]['orig'],'ti':test_pairs[i]['target']}})

## Preprocess

In [18]:
from transformers import AutoTokenizer

checkpoint = "google/mt5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [19]:
source_lang = "en"
target_lang = "ti"
prefix = "translate English to Tigrinya: "


def preprocess_function(examples):
    inputs = [prefix + example['translation'][source_lang] for example in examples]
    targets = [example['translation'][target_lang] for example in examples]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs
def preprocess_functionold(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [20]:
import tensorflow as tf
import pandas as pd
from datasets import Dataset

train_data = Dataset.from_list(train_list)
val_data = Dataset.from_list(val_list)
test_data = Dataset.from_list(test_list)

In [21]:
train_data

Dataset({
    features: ['id', 'translation'],
    num_rows: 826
})

In [22]:
tokenized_train = train_data.map(preprocess_functionold, batched=True)

Map:   0%|          | 0/826 [00:00<?, ? examples/s]

In [23]:
tokenized_val = val_data.map(preprocess_functionold, batched=True)


Map:   0%|          | 0/177 [00:00<?, ? examples/s]

In [25]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google/mt5-base", return_tensors="tf")

## Evaluate

In [26]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00


In [27]:
import evaluate

metric = evaluate.load("bleu")

In [28]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Train

In [29]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [30]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

tf_model.h5:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [31]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_val_set = model.prepare_tf_dataset(
    tokenized_val,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)


In [32]:
import tensorflow as tf
tf.random.set_seed(1234)
model.compile(optimizer=optimizer)  # No loss argument!

In [33]:
from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_val_set)

In [35]:
callbacks = [metric_callback]

In [36]:
tokenizer.get_vocab() #['ዊ']

{'▁heilbrigðis': 195339,
 'ելը': 92954,
 '▁elle': 15679,
 '▁викона': 47642,
 'Marcus': 65847,
 'និយម': 156430,
 'hare': 64157,
 'rollen': 85046,
 '手作り': 125259,
 'հարու': 204010,
 'لترا': 226895,
 'ویس': 52931,
 'anvan': 201227,
 '▁Eh': 36036,
 'Continue': 24970,
 'تصني': 79630,
 'økken': 158721,
 'ukhu': 219826,
 '209)': 227062,
 '▁<extra_id_9>': 250090,
 'loader': 73010,
 'lending': 127746,
 'unar': 32723,
 '▁Guesthouse': 135322,
 'プー': 119792,
 '▁қи': 31848,
 'teori': 115371,
 '2014.03.': 233836,
 '18:12': 151560,
 '▁meme': 34807,
 'dhaan': 198799,
 '▁সং': 9039,
 '▁zielon': 99419,
 'ドイツ': 76115,
 'daniel': 180971,
 'ones': 8317,
 'maskine': 109387,
 '2:42': 141784,
 'üchtig': 201361,
 '▁Hyper': 33408,
 'jaso': 50498,
 'gaism': 218843,
 'tains': 137941,
 'kolt': 113500,
 'acquisition': 78047,
 'женим': 224827,
 'мах': 30423,
 '▁sla': 87575,
 'ndiko': 171316,
 'ectable': 222215,
 'minare': 169874,
 '▁секрет': 74411,
 'likning': 150098,
 'oksy': 191592,
 '0,05': 117344,
 '峁': 241728,
 

In [37]:
model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=60) 

Epoch 1/60
51/51 [==============================] - 59s 339ms/step - loss: 17.8242 - val_loss: 12.0101
Epoch 2/60
51/51 [==============================] - 9s 183ms/step - loss: 14.2294 - val_loss: 10.3682
Epoch 3/60
51/51 [==============================] - 9s 183ms/step - loss: 11.7417 - val_loss: 7.9997
Epoch 4/60
51/51 [==============================] - 9s 184ms/step - loss: 9.8452 - val_loss: 6.4362
Epoch 5/60
51/51 [==============================] - 9s 183ms/step - loss: 8.6240 - val_loss: 5.9047
Epoch 6/60
51/51 [==============================] - 9s 184ms/step - loss: 7.5320 - val_loss: 5.6861
Epoch 7/60
51/51 [==============================] - 9s 183ms/step - loss: 7.1472 - val_loss: 5.5126
Epoch 8/60
51/51 [==============================] - 9s 184ms/step - loss: 6.7874 - val_loss: 5.3804
Epoch 9/60
51/51 [==============================] - 9s 184ms/step - loss: 6.4956 - val_loss: 5.3119
Epoch 10/60
51/51 [==============================] - 9s 185ms/step - loss: 6.4021 - val_loss: 

## Inference

In [38]:
tokenized_test = test_data.map(preprocess_functionold, batched=True)

Map:   0%|          | 0/177 [00:00<?, ? examples/s]

In [39]:
tf_test_set = model.prepare_tf_dataset(
    tokenized_test,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [40]:
train_pairs[0:5]

[{'orig': 'But you shall go to my country and to my relatives and take a wife for my son Isaac.',
  'target': 'ናብ ሃገረይን ናብ ዓሌተይን ኬድካ ደኣ ንወደይ ይስሃቅ ሰበይቲ ኣምጽኣሉ፡ በሎ።'},
 {'orig': 'I will go down now and see whether their deeds are as bad as the reports which have come to me. If not I will know.',
  'target': 'ስለዚ ኸምቲ ናባይ ዝመጸ ጭራሕምራሕ ፈጺሞም ገይሮምዎ ወይ ዘይገበርዎ እንተ ዀይኖም ክፈልጥ፡ ወሪደ ኽርኢ እየ፡ በለ።'},
 {'orig': "Baal Hanan the son of Achbor died and Hadar reigned in his place. The name of his city was Pau. His wife''s name was Mehetabel the daughter of Matred the daughter of Mezahab.",
  'target': 'በዓል-ሓናን ወዲ ኣኽቦር ድማ ሞተ። ሃዳር ከኣ ኣብ ክንዳኡ ነገሰ። ስም ከተማኡ ድማ ጳዑ ነበረ፡ ስም ሰበይቱ ኸኣ ሜሄጣብኤል እዩ፡ ንሳ ጓል ማጥሬድ ጓል ሜ-ዛሃብ እያ።'},
 {'orig': "From now on when you till the ground it won''t yield its strength to you. You shall be a fugitive and a wanderer in the earth.",
  'target': 'ንምድሪ ምስ እትሓርሳ መሊሳ ሓይላ ኣይክትህበካን እያ፡ ኣብ ምድሪ ኰብላልን ቀባሕባሓይን ኩን፡ በለ።'},
 {'orig': 'These are the kings who reigned in the land of Edom before any king reig

In [42]:
from transformers import pipeline



In [43]:
from transformers import AutoTokenizer



In [44]:
from transformers import TFAutoModelForSeq2SeqLM


In [45]:
search_toks = {v:k for k, v in tokenizer.get_vocab().items()}


In [46]:
preds = []
trues = []
for text in test_pairs:
  inputs = tokenizer(text["orig"], return_tensors="tf").input_ids
  outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
  preds.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  trues.append(text["target"])

In [47]:
preds[0:10]

['ያብቆብ ካብተይ በለ ድማ ካብብ ተቐብዓል ኣምላ ናብ ን ዕግዚኣብ ዓይኒ ',
 'ስእዚ፡ ኣብ ኣዋልድማ ምድሪ።',
 'እልግና ክዕንልሕብ',
 ' ከኣ   እተን ኸኣ ደቂ ንሳ ነበረ ሰብኣው ድማ ነበረ።',
 'ነይ ኣምላኽ ብዓብ።',
 'ድማ ንሄር ድማ ኸኣ ራዕል ትወት ነቲ ኣብ ኢ ካብ ድማ ኣብቲ ስግሪት',
 'ሎ፡ እዩ ስኣንቲ ኣብርሃት ከኣ ምድሪ ድማ ድማ ኸኣ ኺረደ።',
 'ብኣቦብኽል ሓደ ኽቡድን።',
 'ንዓመይ',
 '  ፡ ኵሉ ድማ ኣብ ኽእቲ ኽልካ፡ ምድሪ ናብሆ ንሳቶም ድማ ኣብሔርን ተ']

In [49]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=preds, references=trues, max_order = 1)
print(results)

{'bleu': 0.017013478890360652, 'precisions': [0.05451295799821269], 'brevity_penalty': 0.31209971931661595, 'length_ratio': 0.4620148637489678, 'translation_length': 1119, 'reference_length': 2422}
